---
# **7. Монте-Карло симуляция портфеля**

Для каждого актива $i$ в день $j$:

$$r_i(j) = (\mu_i - \frac{1}{2}\sigma_i^2) dt + \sigma_i \sqrt{dt} \epsilon_i(j)$$

где:
- $dt = 1/252$ (один торговый день)
- $\epsilon_i(j) \sim N(0,1)$ — стандартное нормальное
- Поправка $-\frac{1}{2}\sigma_i^2$ — коррекция Ито

### Корреляционная структура

$$\Sigma = L L^T$$

Если $z \sim N(0, I)$, то $\epsilon = Lz \sim N(0, \Sigma)$ имеет нужную корреляцию.

### Портфельная доходность

$$R_p(j) = \mathbf{w}^T \mathbf{r}(j) = \sum_{i=1}^n w_i r_i(j)$$

### Капитал портфеля

$$X_t = X_0 \prod_{j=0}^{t-1} (1 + R_p(j))$$

### Пассивная эволюция весов

$$w_i^{(t)} = \frac{w_i^{(0)} \prod_j (1 + r_i(j))}{\sum_k w_k^{(0)} \prod_j (1 + r_k(j))}$$

Активы с большей доходностью получают больший вес.

---


In [ ]:
def simulate_portfolio_passive(mu, sigma_cholesky, X_0, strategy, asset_names, t_grid,
                               n_paths=5000, random_seed=None):

    if random_seed is not None:
        np.random.seed(random_seed)
        print(f" Установлен random_seed = {random_seed} для воспроизводимости\n")

    mu = np.asarray(mu).flatten()
    sigma_cholesky = np.asarray(sigma_cholesky)
    n_assets = len(mu)

    if sigma_cholesky.shape[0] != n_assets:
        raise ValueError(f"Размерность sigma_cholesky {sigma_cholesky.shape} не совпадает с mu {n_assets}")

    if len(t_grid) < 2:
        raise ValueError(f"t_grid должна иметь минимум 2 элемента, получено {len(t_grid)}")

    dt = t_grid[1] - t_grid[0]
    num_steps = len(t_grid)
    BANKRUPTCY_THRESHOLD = 1e-6

    print(f"  Параметры симуляции:")
    print(f"   n_paths:        {n_paths:,}")
    print(f"   num_steps:      {num_steps}")
    print(f"   dt (шаг):       {dt:.4f} лет")
    print(f"   T (горизонт):   {t_grid[-1] - t_grid[0]:.2f} лет")
    print(f"   random_seed:    {random_seed if random_seed is not None else 'None (случайный)'}\n")

    X_paths = np.zeros((n_paths, num_steps))
    X_paths[:, 0] = X_0

    weights_history = np.zeros((n_paths, num_steps, n_assets))

    initial_weights = strategy.get_optimal_weights()

    initial_sum = np.sum(initial_weights)
    if initial_sum < 0.99:
        print(f"  Внимание: Начальные веса суммируются в {initial_sum:.4f}.")
        if strategy.normalize_weights:
            print(f"   (Это нормально: остаток инвестируется в risk-free asset)")
        print()

    weights_history[:, 0, :] = initial_weights[np.newaxis, :]

    current_positions = np.tile(initial_weights * X_0, (n_paths, 1))

    dW = np.random.normal(0, np.sqrt(dt), (n_paths, n_assets, num_steps - 1))

    variances = np.diag(sigma_cholesky @ sigma_cholesky.T)
    variance_correction = 0.5 * variances

    for step in range(num_steps - 1):

        asset_drifts = (mu - variance_correction) * dt

        asset_diffusions = np.dot(dW[:, :, step], sigma_cholesky.T)

        log_returns = asset_drifts + asset_diffusions

        current_positions = current_positions * np.exp(log_returns)

        current_positions = np.maximum(current_positions, 0)

        X_new = np.sum(current_positions, axis=1)

        bankrupt_mask = X_new < BANKRUPTCY_THRESHOLD
        X_new[bankrupt_mask] = BANKRUPTCY_THRESHOLD
        current_positions[bankrupt_mask, :] = 0

        X_paths[:, step + 1] = X_new

        with np.errstate(divide='ignore', invalid='ignore'):
            new_weights = current_positions / X_new[:, np.newaxis]
            new_weights[bankrupt_mask, :] = 0

        weights_history[:, step + 1, :] = new_weights

    final_weights_mean = np.mean(weights_history[:, -1, :], axis=0)
    final_weights_std = np.std(weights_history[:, -1, :], axis=0, ddof=1)
    sum_weights = np.sum(final_weights_mean)

    final_capitals = X_paths[:, -1]
    bankruptcy_count = np.sum(final_capitals < BANKRUPTCY_THRESHOLD * 10)

    print(f" Симуляция завершена\n")
    print(f"   Траекторий:       {n_paths:,}")
    print(f"   Шаг времени:      {dt:.4f} лет")

    if bankruptcy_count > 0:
        print(f"     Банкротств:     {bankruptcy_count:,} ({bankruptcy_count/n_paths*100:.2f}%)")
    else:
        print(f"    Банкротств:     0")

    print(f"\n Средние веса в конце (Σ = {sum_weights:.4f}):")
    for i, name in enumerate(asset_names):
        emoji = "🟢" if final_weights_mean[i] > 0.01 else "⚪"
        print(f"   {emoji} {name:5s}: {final_weights_mean[i]*100:>6.2f}% ± {final_weights_std[i]*100:>5.2f}%")


    return X_paths, weights_history, t_grid